# SpotOptim Demonstrations

This notebook demonstrates the usage of the SpotOptim optimizer for various optimization problems.

## Example 1: 2-Dimensional Rosenbrock Function

This example shows basic optimization on the classic 2D Rosenbrock function.

In [ ]:
import numpy as np
from spotoptim import SpotOptim

# Define Rosenbrock function
def rosenbrock(X):
    """Rosenbrock function for optimization."""
    X = np.atleast_2d(X)
    x = X[:, 0]
    y = X[:, 1]
    return (1 - x)**2 + 100 * (y - x**2)**2

# Set up bounds for 2D problem
bounds = [(-2, 2), (-2, 2)]

# Create optimizer
optimizer = SpotOptim(
    fun=rosenbrock,
    bounds=bounds,
    max_iter=50,
    n_initial=5,
    acquisition='ei',
    seed=42,
    verbose=False
)

# Run optimization
result = optimizer.optimize()

# Print results
print("\n" + "="*50)
print("Optimization Results")
print("="*50)
print(f"Best point found: {result.x}")
print(f"Best function value: {result.fun:.6f}")
print(f"Number of function evaluations: {result.nfev}")
print(f"Number of iterations: {result.nit}")
print(f"Success: {result.success}")
print(f"Message: {result.message}")
print("\nTrue optimum: [1, 1] with f(x) = 0")

## Example 2: 6-Dimensional Rosenbrock Function

This example demonstrates optimization of the 6-dimensional Rosenbrock function with a budget of 100 total function evaluations.

In [ ]:
import numpy as np
from spotoptim import SpotOptim

# Define 6-dimensional Rosenbrock function
def rosenbrock_6d(X):
    """
    6-dimensional Rosenbrock function for optimization.
    
    The Rosenbrock function is defined as:
    f(x) = sum_{i=1}^{n-1} [100*(x_{i+1} - x_i^2)^2 + (1 - x_i)^2]
    
    Global minimum: f(1, 1, 1, 1, 1, 1) = 0
    """
    X = np.atleast_2d(X)
    n_samples, n_dim = X.shape
    
    result = np.zeros(n_samples)
    for i in range(n_samples):
        x = X[i]
        total = 0
        for j in range(n_dim - 1):
            total += 100 * (x[j+1] - x[j]**2)**2 + (1 - x[j])**2
        result[i] = total
    
    return result

# Set up bounds for 6D problem
# Typical search domain for Rosenbrock is [-5, 10] for each dimension
bounds_6d = [(-2, 2)] * 6

# Budget: 100 total function evaluations
# Split into initial design and optimization iterations
n_total = 100
n_initial = 6  # Initial Latin Hypercube Design points
max_iter = n_total - n_initial   # Optimization iterations: 6 + 94 = 100 total evaluations

print("="*60)
print("6D Rosenbrock Function Optimization")
print("="*60)
print(f"Problem dimension: 6")
print(f"Search bounds: [-2, 2] for each dimension")
print(f"Total budget: {n_initial + max_iter} function evaluations")
print(f"  - Initial design: {n_initial} points")
print(f"  - Optimization iterations: {max_iter}")
print(f"Global optimum: x* = [1, 1, 1, 1, 1, 1], f(x*) = 0")
print("="*60)

# Create optimizer with Expected Improvement acquisition
optimizer_6d = SpotOptim(
    fun=rosenbrock_6d,
    bounds=bounds_6d,
    max_iter=max_iter,
    n_initial=n_initial,
    acquisition='y',  # Expected Improvement
    seed=42,
    verbose=True
)

# Run optimization
print("\nStarting optimization...\n")
result_6d = optimizer_6d.optimize()

# Print final results
print("\n" + "="*60)
print("Final Optimization Results")
print("="*60)
print(f"Best point found: {result_6d.x}")
print(f"Best function value: {result_6d.fun:.6e}")
print(f"Number of function evaluations: {result_6d.nfev}")
print(f"Number of iterations: {result_6d.nit}")
print(f"Success: {result_6d.success}")

# Calculate distance from true optimum
true_optimum = np.ones(6)
distance_to_optimum = np.linalg.norm(result_6d.x - true_optimum)
print(f"\nDistance to true optimum [1,1,1,1,1,1]: {distance_to_optimum:.6f}")

# Show improvement over initial design
initial_best = np.min(result_6d.y[:n_initial])
final_best = result_6d.fun
improvement = initial_best - final_best
improvement_pct = (improvement / initial_best) * 100

print(f"\nImprovement analysis:")
print(f"  Best initial value: {initial_best:.6e}")
print(f"  Final best value: {final_best:.6e}")
print(f"  Absolute improvement: {improvement:.6e}")
print(f"  Relative improvement: {improvement_pct:.2f}%")
print("="*60)

## Example 3: Using Kriging Surrogate

This example demonstrates how to use the Kriging surrogate model instead of the default Gaussian Process from scikit-learn.

In [ ]:
import numpy as np
from spotoptim import SpotOptim, Kriging

# Define 2D Sphere function
def sphere_2d(X):
    """
    2D Sphere function: f(x, y) = x^2 + y^2
    Global minimum: f(0, 0) = 0
    """
    X = np.atleast_2d(X)
    return np.sum(X**2, axis=1)

# Set up bounds
bounds_sphere = [(-5, 5), (-5, 5)]

print("="*60)
print("Optimization with Kriging Surrogate")
print("="*60)
print("Function: 2D Sphere (f(x,y) = x² + y²)")
print("Search bounds: [-5, 5] for each dimension")
print("Global optimum: x* = [0, 0], f(x*) = 0")
print("="*60)

# Create Kriging surrogate with custom parameters
kriging_surrogate = Kriging(
    noise=1e-6,          # Small regularization
    min_theta=-3.0,      # Bounds for length scale optimization
    max_theta=2.0,
    seed=42
)

# Create optimizer with Kriging surrogate
optimizer_kriging = SpotOptim(
    fun=sphere_2d,
    bounds=bounds_sphere,
    max_iter=20,
    n_initial=10,
    surrogate=kriging_surrogate,  # Use Kriging instead of default GP
    acquisition='ei',
    seed=42,
    verbose=True
)

# Run optimization
print("\nStarting optimization with Kriging surrogate...\n")
result_kriging = optimizer_kriging.optimize()

# Print results
print("\n" + "="*60)
print("Final Results (Kriging Surrogate)")
print("="*60)
print(f"Best point found: {result_kriging.x}")
print(f"Best function value: {result_kriging.fun:.6e}")
print(f"Distance to optimum [0, 0]: {np.linalg.norm(result_kriging.x):.6f}")
print(f"Number of function evaluations: {result_kriging.nfev}")
print(f"Number of iterations: {result_kriging.nit}")

# Compare with default GP surrogate
print("\n" + "="*60)
print("Comparison: Running with Default GP Surrogate")
print("="*60)

optimizer_gp = SpotOptim(
    fun=sphere_2d,
    bounds=bounds_sphere,
    max_iter=20,
    n_initial=10,
    acquisition='ei',
    seed=42,
    verbose=False  # Quiet for comparison
)

result_gp = optimizer_gp.optimize()

print(f"GP Best value: {result_gp.fun:.6e}")
print(f"GP Distance to optimum: {np.linalg.norm(result_gp.x):.6f}")
print(f"\nKriging Best value: {result_kriging.fun:.6e}")
print(f"Kriging Distance to optimum: {np.linalg.norm(result_kriging.x):.6f}")

print("\n" + "="*60)
print("Summary")
print("="*60)
print("Both Kriging and GP surrogates successfully found the optimum.")
print("Kriging offers a simplified, self-contained surrogate model")
print("that can be customized with different hyperparameters.")
print("="*60)

## Example 4: Visualizing the Surrogate Model

This example demonstrates the `plot_surrogate()` method, which visualizes the fitted surrogate model and its uncertainty. We'll use a 2D sphere function to clearly show the model's predictions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spotoptim import SpotOptim

# Define a 2D sphere function
def sphere_2d(X):
    """Simple 2D sphere function: f(x) = x1^2 + x2^2"""
    X = np.atleast_2d(X)
    return np.sum(X**2, axis=1)

# Set up the optimization problem
bounds = [(-5, 5), (-5, 5)]

# Create optimizer with limited iterations to see exploration
optimizer = SpotOptim(
    fun=sphere_2d,
    bounds=bounds,
    max_iter=15,
    n_initial=8,
    acquisition='ei',
    seed=42,
    verbose=True
)

# Run optimization
print("Running optimization...")
result = optimizer.optimize()

print("\n" + "="*50)
print("Optimization Results")
print("="*50)
print(f"Best point found: {result.x}")
print(f"Best function value: {result.fun:.6f}")
print(f"Number of function evaluations: {result.nfev}")
print(f"True optimum: [0, 0] with f(x) = 0")
print(f"Distance to optimum: {np.linalg.norm(result.x):.6f}")

# Visualize the surrogate model
print("\nGenerating surrogate model visualization...")
optimizer.plot_surrogate(
    i=0, j=1,
    var_name=['x1', 'x2'],
    add_points=True,
    cmap='viridis',
    show=True
)

### Visualizing Higher-Dimensional Problems

For problems with more than 2 dimensions, `plot_surrogate()` shows a 2D slice by fixing all other dimensions at their mean values. This example demonstrates visualization of dimensions 0 and 2 from a 4D problem.

In [ ]:
import numpy as np
from spotoptim import SpotOptim

# Define a 4D sphere function
def sphere_4d(X):
    """4D sphere function: f(x) = sum(xi^2)"""
    X = np.atleast_2d(X)
    return np.sum(X**2, axis=1)

# Set up bounds for 4D problem
bounds = [(-3, 3), (-3, 3), (-3, 3), (-3, 3)]

# Create optimizer
optimizer_4d = SpotOptim(
    fun=sphere_4d,
    bounds=bounds,
    max_iter=20,
    n_initial=10,
    acquisition='ei',
    seed=123,
    verbose=False
)

# Run optimization
result_4d = optimizer_4d.optimize()

print("4D Optimization Results:")
print(f"Best point: {result_4d.x}")
print(f"Best value: {result_4d.fun:.6f}")
print(f"Distance to optimum: {np.linalg.norm(result_4d.x):.6f}")

# Visualize dimensions 0 and 2 (dimensions 1 and 3 fixed at mean)
print("\nVisualizing dimensions 0 and 2 (others fixed at mean values)...")
optimizer_4d.plot_surrogate(
    i=0, j=2,
    var_name=['x0', 'x1', 'x2', 'x3'],
    add_points=True,
    cmap='coolwarm',
    contour_levels=25,
    show=True
)

# You can also visualize other dimension pairs
print("\nVisualizing dimensions 1 and 3...")
optimizer_4d.plot_surrogate(
    i=1, j=3,
    var_name=['x0', 'x1', 'x2', 'x3'],
    add_points=True,
    cmap='plasma',
    show=True
)

In [ ]:
from spotoptim import SpotOptim
opt = SpotOptim(fun=lambda X: np.sum(X**2, axis=1),
                 bounds=[(-5, 5), (-5, 5)],
                 n_initial=10)
X0 = opt._generate_initial_design()
X0.shape

In [ ]:
import spotoptim as so
opt = so.SpotOptim(fun=lambda X: np.sum(X**2, axis=1),
                   bounds=[(-5, 5), (-5, 5)],
                   n_initial=10)
X0 = opt._generate_initial_design()
X0.shape

## Success Rate Tracking

In [ ]:
import numpy as np
from spotoptim import SpotOptim

def rosenbrock(X):
    """Rosenbrock function - challenging optimization problem"""
    x = X[:, 0]
    y = X[:, 1]
    return (1 - x)**2 + 100 * (y - x**2)**2

# Run optimization with periodic success rate checks
optimizer = SpotOptim(
    fun=rosenbrock,
    bounds=[(-2, 2), (-2, 2)],
    max_iter=200,
    n_initial=20,
    tensorboard_log=True,
    tensorboard_clean=True,
    seed=42
)

result = optimizer.optimize()

# Analyze final success rate
print(f"\nOptimization Results:")
print(f"Best value: {result.fun:.6f}")
print(f"Total evaluations: {optimizer.counter}")
print(f"Final success rate: {optimizer.success_rate:.2%}")

# Interpret the result
if optimizer.success_rate > 0.5:
    print("→ High success rate: Optimization is still making good progress")
elif optimizer.success_rate > 0.2:
    print("→ Medium success rate: Approaching convergence")
else:
    print("→ Low success rate: Optimization has likely converged")

# Hyperparameter Tuning Example

In [1]:
import numpy as np
import torch
import torch.nn as nn
from spotoptim import SpotOptim
from spotoptim.data import get_diabetes_dataloaders
from spotoptim.nn.linear_regressor import LinearRegressor

def evaluate_model(X):
    """Objective function for SpotOptim.

    Args:
        X: Array of hyperparameters [lr, l1, num_hidden_layers]

    Returns:
        Array of validation losses
    """
    results = []

    for params in X:
        lr, l1, num_hidden_layers = params
        lr = 10 ** lr  # Log scale for learning rate
        l1 = int(l1)
        num_hidden_layers = int(num_hidden_layers)

        # Load data
        train_loader, test_loader, _ = get_diabetes_dataloaders(
            test_size=0.2,
            batch_size=32,
            random_state=42
        )

        # Create model
        model = LinearRegressor(
            input_dim=10,
            output_dim=1,
            l1=l1,
            num_hidden_layers=num_hidden_layers,
            activation="ReLU"
        )

        # Train briefly
        criterion = nn.MSELoss()
        optimizer = model.get_optimizer("Adam", lr=lr)

        num_epochs = 50
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                predictions = model(batch_X)
                loss = criterion(predictions, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Evaluate
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                predictions = model(batch_X)
                loss = criterion(predictions, batch_y)
                test_loss += loss.item()

        results.append(test_loss / len(test_loader))

    return np.array(results)

# Optimize hyperparameters
optimizer = SpotOptim(
    fun=evaluate_model,
    bounds=[
        (-4, -2),   # log10(lr): 0.0001 to 0.01
        (16, 128),  # l1: number of neurons
        (0, 4)      # num_hidden_layers
    ],
    var_type=["num", "int", "int"],
    max_iter=100,
    n_initial=10,
    seed=42,
    verbose=True,
    tensorboard_clean=True,
    tensorboard_log=True
)

result = optimizer.optimize()
print(f"Best hyperparameters found:")
print(f"  Learning rate: {10**result.x[0]:.6f}")
print(f"  Hidden neurons (l1): {int(result.x[1])}")
print(f"  Hidden layers: {int(result.x[2])}")
print(f"  Best MSE: {result.fun:.4f}")

Removed old TensorBoard logs: runs/spotoptim_20251115_210430
Cleaned 1 old TensorBoard log directory
TensorBoard logging enabled: runs/spotoptim_20251115_232906
Initial best: f(x) = 2719.988281
Iteration 1: f(x) = 2949.463867
Initial best: f(x) = 2719.988281
Iteration 1: f(x) = 2949.463867
Iteration 2: f(x) = 3095.499674
Iteration 2: f(x) = 3095.499674
Iteration 3: f(x) = 4099.496582
Iteration 3: f(x) = 4099.496582
Iteration 4: f(x) = 3722.035156
Iteration 4: f(x) = 3722.035156
Iteration 5: f(x) = 2890.413737
Iteration 6: f(x) = 2806.675618
Iteration 5: f(x) = 2890.413737
Iteration 6: f(x) = 2806.675618
Iteration 7: f(x) = 3018.945719
Iteration 8: f(x) = 3067.457194
Iteration 7: f(x) = 3018.945719
Iteration 8: f(x) = 3067.457194
Iteration 9: f(x) = 3697.912191
Iteration 9: f(x) = 3697.912191
Iteration 10: f(x) = 2997.858073
Iteration 10: f(x) = 2997.858073
Iteration 11: f(x) = 3101.482666
Iteration 12: f(x) = 3218.051514
Iteration 11: f(x) = 3101.482666
Iteration 12: f(x) = 3218.051514

In [ ]:
optimizer.plot_surrogate(i=0, j=1, var_name=['log10(lr)', 'l1', 'num_hidden_layers'], add_points=True, show=True)